In [15]:
import pandas as pd
import numpy as np

from census import Census
import requests
from ipynb.fs.defs.census_data_acquisition import get_census_tract, get_acs_data_for_county_tracts
from ipynb.fs.defs.census_and_restaurant_data_withEDA import adjoin_census

In [4]:
with open("/Users/dominiquekemp/Documents/GitHub/'Will It Restaurant?'/census_APIkey.csv", "r") as file:
    CENSUS_API_KEY = file.read()

c = Census(CENSUS_API_KEY)

In [5]:
#load the test dataset

rd_test = pd.read_csv("../'Will It Restaurant?'/test_data.csv")

In [7]:
rd_test.range.unique()

array(['$', '$$', '$$$', '0', '$$$$'], dtype=object)

In [8]:
#a preliminary: apply sign map to price range column
map = {'$': '1', '$$': '2', '$$$': '3', '$$$$': 4}

rd_test['range'] = rd_test.range.map(map)

In [10]:
#get all tracts
all_tracts = []
for i in range(len(rd_test)):
    lat, lon = rd_test[['latitude', 'longitude']].values[i]
    all_tracts.append(get_census_tract(lat, lon))

#consolidate tracts
unique_tracts = []
tract_nos = []
for tract in all_tracts:
    tract_num = tract['tract']
    if tract_num not in tract_nos:
        unique_tracts.append(tract)
        tract_nos.append(tract['tract'])

#extract info from tracts needed for get_acs_... function
counties = []
tracts = []
for tract in unique_tracts:
    counties.append(tract['county'])
    tracts.append(tract['tract'])
    

In [11]:
#obtain the census data
variables_to_get = ("B19013_001E", 'B17001_002E', 'B17001_001E', 'B15003_001E', 'B15003_022E', 'B02001_001E', 
                                       'B02001_002E', 'B02001_003E', 'B02001_005E', 'B03003_003E', 'B05002_013E', 
                                       'B05002_001E', 'B01002_001E', 'B07003_017E', 'B07003_001E')

acs_data = get_acs_data_for_county_tracts(c, counties, tracts, variables_to_get)


In [12]:
#clean it of eccentric values
acs_data.replace(-666666666.0, np.nan, inplace=True)

i=0
while i < len(acs_data.index):
    boolean_for_zero_values = (acs_data.iloc[i, list(range(1,12)) + [13,14]] == 0)
    if boolean_for_zero_values.all():
        acs_data.drop(acs_data.index[i], inplace = True)
    else:
        i += 1

#reorganize the census dataframe with relabeled columns and percentages for the values
census_dict = {"B19013_001E":'Median Household Income', "B17001_002E": "Poverty Rate Num", "B17001_001E": "Poverty Rate Denom", "B15003_022E": 'Education Level Num', "B15003_001E": "Education Level Denom", "B02001_001E": "Total Population",
                 "B02001_002E": "% White", "B02001_003E": "% Black", "B02001_005E": "% Asian", "B03003_003E": "% Hispanic/Latino", "B05002_013E": "Foreign Born Immigrant Num", 
                "B05002_001E": "Foreign Born Immigrant Denom", "B01002_001E": "Median Age", "B07003_017E": "Neighborhood Turnover Num", "B07003_001E": "Neighborhood Turnover Denom"}

acs_data.rename(columns = census_dict, inplace = True)

perc_list = [['Poverty Rate Num', 'Poverty Rate Denom'], ['Education Level Num', 'Education Level Denom'], ['% White', 'Total Population'],
             ['% Black', 'Total Population'], ['% Asian', 'Total Population'], ['% Hispanic/Latino', 'Total Population'], ['Foreign Born Immigrant Num', 'Foreign Born Immigrant Denom'],
             ['Neighborhood Turnover Num', 'Neighborhood Turnover Denom']]

for ratio in perc_list:
    acs_data[ratio[0]] = acs_data[ratio[0]]/acs_data[ratio[1]]
    if ratio[1] != 'Total Population':
        acs_data.drop(ratio[1], axis = 1, inplace = True)

acs_data.rename(columns = {'Poverty Rate Num': "Poverty Rate %", 'Education Level Num': 'Education Level %', 
                           'Foreign Born Immigrant Num': 'Foreign Born Immigrant %', 
                           'Neighborhood Turnover Num': 'Neighborhood Turnover %'}, inplace = True)
    

acs_data.sort_values(by = "tract", inplace = True)
 

In [13]:
acs_data

,Median Household Income,Poverty Rate %,Education Level %,Total Population,% White,% Black,% Asian,% Hispanic/Latino,Foreign Born Immigrant %,Median Age,Neighborhood Turnover %,state,county,tract
5,104052.0,0.022954,0.461057,2004.0,0.839820,0.013972,0.078343,0.172655,0.131737,30.9,0.005814,42,101,000101
56,91944.0,0.058866,0.450782,2752.0,0.912427,0.029797,0.027616,0.048692,0.085392,33.5,0.000000,42,101,000102
10,91067.0,0.187822,0.383935,3414.0,0.311365,0.065905,0.553017,0.050088,0.403046,44.3,0.002058,42,101,000200
50,86782.0,0.095180,0.333099,3566.0,0.636287,0.061133,0.172462,0.074874,0.199663,36.4,0.012972,42,101,000300
86,67188.0,0.210506,0.418533,2573.0,0.619899,0.145744,0.184609,0.033813,0.322192,34.6,0.011028,42,101,000401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,45313.0,0.273478,0.079236,4492.0,0.175868,0.632012,0.116874,0.094613,0.297640,32.7,0.000000,42,045,400402
35,67909.0,0.212534,0.148634,5952.0,0.256720,0.663642,0.013777,0.047547,0.133233,33.3,0.000000,42,045,402800
219,42033.0,0.250302,0.042798,3340.0,0.028443,0.947605,0.000000,0.019461,0.031138,34.3,0.000000,42,045,402900
70,44149.0,0.162615,0.131498,2387.0,0.886468,0.021785,0.000000,0.069124,0.000838,36.2,0.000000,42,045,403702


In [16]:
#append column of census tract numbers to restaurant dataframe
rd_test["census_tract"] = [tract['tract'] for tract in all_tracts]

#adjoin census data
rd_test = adjoin_census(rd_test, acs_data)

In [18]:
#write to file
rd_test.to_csv("../'Will It Restaurant?'/test_data.csv", index = False)